# Описание

Я фаюнтюнила RuGPT3Small для стихов ([датасет](https://www.kaggle.com/grafstor/19-000-russian-poems)). Цель -- генерация текста, похожено на стихи; таргет -- смещённые последовательности; метрика -- перплексия.

Выбрала из датасета стихи Фета, т.к. их было среднее количество (изначально хотела Цветаеву, но её было слишком мало и разброс в длине большой) примерно одинаковой длины. 

Изначально опиралась на скрипты сбера: 
* [их репозиторий](https://github.com/sberbank-ai/ru-gpts) 
* [их тетрадка с примером файнтюнинга](https://colab.research.google.com/github/sberbank-ai/ru-gpts/blob/master/examples/Finetune_ruGPT3Small.ipynb#scrollTo=5vL07XFvsBBU) 
* [скрипт для обучения, который там запускается](https://github.com/sberbank-ai/ru-gpts/blob/master/pretrain_transformers.py) 
* [скрипт для генерации, который там запускается](https://github.com/sberbank-ai/ru-gpts/blob/master/generate_transformers.py)

Бейзлайном, по идее, должна быть генерация текстов по тем же данным самой оригинальной RuGPT3Small. 

Я, к сожалению, поздно начала, поэтому пришлось брать маленький корпус, чтобы обучалось побыстрее. В итоге не обучилось толком ничего, предсказания на текст не похожи, а перплексия 49,5 (в критериях проекта было указано хотя бы ниже 50, так что я вроде как справилась, но на самом деле не особо). 

Кроме того, возникла проблема с подготовкой данных: в скрипте сбера, если я правильно в нём разобралась, я обнаружила, что у них метки -- это копия инпута, то есть они не смещены на один таймстеп вперёд. Покопалась в скрипте мегатрона, котрый был исходником для сбера, но там метки смещены. 
* [сберовский](https://github.com/sberbank-ai/ru-gpts/blob/8234af5cce96cd766f07a461ed5d44957649376e/pretrain_transformers.py#L173) 
* [мегатрон в репе сбера](https://github.com/sberbank-ai/ru-gpts/blob/8234af5cce96cd766f07a461ed5d44957649376e/pretrain_megatron.py#L261)
* [мегатрон в их собственной репе](https://github.com/microsoft/DeepSpeedExamples/blob/78d69cb2f89a27b1e9b072df8c3e47d00c024fdc/Megatron-LM/pretrain_gpt2.py#L265)

Решила в итоге делать, как учили и как у мегатрона, но так и не уверена в правильности выбора. При генерации на тесте через model.generate выход оказался не смещённым тоже.

Столкнулась со странным делом: когда я запускала генерацию на тесте через model.generate, всё генерилось точ-в-точь. При попытке написать генерацию самостоятельно, получался бред, не похожий на текст. Кажется, где-то в одном из этих вариантов закралась и спряталась от меня ошибка.

Я попробовала несколько методов генерации: 
* greedy sampling: семплируется самый вероятный токен
* top-k sampling: вероятности токенов перераспределяются между k самыми вероятными, семплируется с этой вероятностью
* семплинг k самых вероятных токенов с равной вероятностью  
* nucleus sampling: вероятности токенов перераспределяются между минимальным набором токенов, вероятность которых в сумме превышает порог p, семплируется с этой вероятностью

Первый выдавал сплошные запятые, два следующих смогли учесть обилие окончаний "ой" и "ою", а последний даже старался составлять слова, хотя повтрял их и склеивал без пробелов.

Модель предсказывала сразу последовательности, но, как я уже сказала, предсказывался там такой бред, что я подумала, что подавать их на вход модели обратно уже бессмысленно. Никаких eos там не предсказывалось. 

Возможно, сыграла роль ещё особенность домена: в стихах много нечастотных слов, слов с нечастотной или устаревшей формой, и в итоге при токенизации мало что объединялось в слова, в основном были кусочки, которые модель пыталась соединять. С этим можно было бы побороться разными методами доменной адаптации, если уж не увеличивать датасет, ну или хотя бы запилить beam search или составить параллельный вокаб из слов, а не кусочков, и генерить то, что в него укладывается, но я, к сожалению, не успела.

Но теперь я знаю, как приспособить большие модельки под себя, если понадобится, хотя раньше это казалось сложным, почитала про разные методы генерации и написала несколько легко переносимых в другой код функций для этого, котрыми потом воспользовалась в домашке по seqseq. Так что можно сказать, что всё было не зря)

# Установка

In [1]:
!wget -nc https://raw.githubusercontent.com/AnnaSafaryan/DLProject/main/requirements.txt

--2020-12-26 19:45:58--  https://raw.githubusercontent.com/AnnaSafaryan/DLProject/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     130  --.-KB/s    in 0s      

2020-12-26 19:45:59 (8.13 MB/s) - ‘requirements.txt’ saved [130/130]



In [2]:
!pip install -r requirements.txt

     |████████████████████████████████| 1.4MB 8.5MB/s 
     |████████████████████████████████| 1.1MB 30.3MB/s 
     |████████████████████████████████| 133kB 54.5MB/s 
     |████████████████████████████████| 696kB 44.1MB/s 
     |████████████████████████████████| 573kB 55.9MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 5.9MB 53.5MB/s 
     |████████████████████████████████| 3.7MB 51.2MB/s 
     |████████████████████████████████| 890kB 53.1MB/s 
     |████████████████████████████████| 552kB 43.8MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434677 sha256=2871f2ed02ae2d4a31d6e98f75f8af58a4903164401b531ef39d78a5b0d98271
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=20aacf5411a35961293258e3935cc5b2102b95baf990a1c88830ddc1a3466bd3
  Stored in directory: /roo

# Импорты

In [3]:
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import random
from collections import defaultdict
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import (
    AdamW,
    AutoModelWithLMHead,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)

import warnings
warnings.filterwarnings('ignore')

# Данные

In [4]:
!wget -nc https://raw.githubusercontent.com/AnnaSafaryan/DLProject/main/poems.csv

--2020-12-26 19:46:19--  https://raw.githubusercontent.com/AnnaSafaryan/DLProject/main/poems.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42973998 (41M) [text/plain]
Saving to: ‘poems.csv’

poems.csv           100%[===================>]  40.98M   119MB/s    in 0.3s    

2020-12-26 19:46:19 (119 MB/s) - ‘poems.csv’ saved [42973998/42973998]



In [5]:
poems = pd.read_csv('poems.csv')
poems

,writer,poem,text
0,Лермонтов Михаил Юрьевич,Любовь мертвеца,NaN
1,Лермонтов Михаил Юрьевич,На серебряные шпоры…,На серебряные шпоры\nЯ в раздумии гляжу;\nЗа т...
2,Лермонтов Михаил Юрьевич,Вид гор из степей Козлова,Пилигрим\nАллах ли там среди пустыни\nЗастывши...
3,Лермонтов Михаил Юрьевич,"К (О, не скрывай! Ты плакала об нем…)","О, не скрывай! Ты плакала об нем –\nИ я его лю..."
4,Лермонтов Михаил Юрьевич,"Жалобы турка (письмо к другу, иностранцу)","Ты знал ли дикий край, под знойными лучами,\nГ..."
...,...,...,...
19311,Владислав Фелицианович Ходасевич,Прогулка,"Хорошо, что в этом мире\nЕсть магические ночи,..."
19312,Владислав Фелицианович Ходасевич,Пробочка,Пробочка над крепким иодом!\nКак ты скоро пере...
19313,Владислав Фелицианович Ходасевич,"Друзья, друзья! Быть может, скоро…","Друзья, друзья! Быть может, скоро —\nИ не во с..."
19314,Владислав Фелицианович Ходасевич,"Увы, дитя! Душе неутоленной…","Увы, дитя! Душе неутоленной\nНе снишься ль ты ..."


In [6]:
poems.dropna(inplace=True)
poems

,writer,poem,text
1,Лермонтов Михаил Юрьевич,На серебряные шпоры…,На серебряные шпоры\nЯ в раздумии гляжу;\nЗа т...
2,Лермонтов Михаил Юрьевич,Вид гор из степей Козлова,Пилигрим\nАллах ли там среди пустыни\nЗастывши...
3,Лермонтов Михаил Юрьевич,"К (О, не скрывай! Ты плакала об нем…)","О, не скрывай! Ты плакала об нем –\nИ я его лю..."
4,Лермонтов Михаил Юрьевич,"Жалобы турка (письмо к другу, иностранцу)","Ты знал ли дикий край, под знойными лучами,\nГ..."
5,Лермонтов Михаил Юрьевич,К кн. Л. Г-ой,Когда ты холодно внимаешь\nРассказам горести ч...
...,...,...,...
19311,Владислав Фелицианович Ходасевич,Прогулка,"Хорошо, что в этом мире\nЕсть магические ночи,..."
19312,Владислав Фелицианович Ходасевич,Пробочка,Пробочка над крепким иодом!\nКак ты скоро пере...
19313,Владислав Фелицианович Ходасевич,"Друзья, друзья! Быть может, скоро…","Друзья, друзья! Быть может, скоро —\nИ не во с..."
19314,Владислав Фелицианович Ходасевич,"Увы, дитя! Душе неутоленной…","Увы, дитя! Душе неутоленной\nНе снишься ль ты ..."


In [7]:
poems.groupby('writer').count()
# print(poems.writer.unique())

,poem,text
writer,,
Агнивцев Николай Яковлевич,148,148
Аделаида Казимировна Герцык,351,351
Александр Иванович Одоевский,82,82
Александр Николаевич Радищев,20,20
Александр Петрович Сумароков,1520,1520
Алексей Васильевич Кольцов,211,211
Алексей Николаевич Апухтин,334,334
Андрей Белый,394,394
Анненский Иннокентий Федорович,278,278


In [8]:
corpus = poems.where(poems['writer'] == 'Фет Афанасий Афанасьевич').dropna().text.values
print(len(corpus))

888


In [9]:
# пожертвуем переносами строк, но так последовательнотси получатся покороче, и меньше памяти потребуется
train_corpus, test_corpus = train_test_split([text.replace('\n', ' ') for text in corpus], test_size=0.2, shuffle=True, random_state=42)
print(len(train_corpus), len(test_corpus))

710 178


# Датасет

In [10]:
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

In [11]:
print(len(tokenizer.get_vocab()), [tokenizer.decode(id) for id in tokenizer.get_vocab().values()])

50257 ['<pad>', '<s>', '</s>', '<unk>', '<mask>', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '\x00', '

In [12]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [13]:
tokenizer.add_special_tokens({'bos_token': '<s>',
                              'eos_token': '</s>',
                              'unk_token': '<unk>',
                              'pad_token': '<pad>'})

0

In [14]:
unks = set(tokenizer.tokenize('\n'.join(corpus))) - set(tokenizer.get_vocab())
print(len(unks), unks)

0 set()


In [15]:
print([tokenizer.decode(id) for id in tokenizer.encode(train_corpus[0].lower())])
print([tokenizer.decode(id) for id in tokenizer.encode(test_corpus[0].lower())])

['была', ' пора', ' —', ' своей', ' иг', 'рою', ',', ' своею', ' ри', 'з', 'ою', ' ст', 'ально', 'ю', ' морской', ' простор', ' меня', ' плен', 'ял', ';', ' я', ' дорож', 'ил', ' и', ' в', ' ти', 'шь', ' и', ' в', ' бури', ' то', ' нег', 'ой', ' та', 'ющей', ' лаз', 'ури', ',', ' то', ' пен', 'ой', ' у', ' прибреж', 'ных', ' скал', '.', ' но', ' вот', ',', ' о', ' море', ',', ' властью', ' тайной', ' не', ' всё', ' мне', ' мил', ' твой', ' блеск', ' случай', 'ный', ' и', ' в', ' душу', ' прос', 'ится', ' мою', ';', ' див', 'ясь', ' кра', 'се', ' жесток', 'овый', 'ной', ',', ' я', ' перед', ' мощи', 'ю', ' стихий', 'ной', ' в', ' священ', 'ном', ' треп', 'ете', ' стою', '.']
['уж', ' вечер', ' над', 'вину', 'ться', ' хочет', ',', ' туман', ' над', ' волнами', ' растет', ',', ' таин', 'ственно', ' море', ' рок', 'очет', ' и', ' что', '-', 'то', ',', ' б', 'еле', 'ясь', ',', ' встает', '.', ' из', ' волн', ' поднимается', ' ф', 'ея', ' и', ' села', ' со', ' мной', ' у', ' зы', 'бей', '.',

In [16]:
class TextDataset(Dataset):
    def __init__(self, data, tokenizer, seq_len=None, length_percentile=90):
        
        if not seq_len: # если не задали длину, вычисляем
            lengths = [len(tokenizer.tokenize(sample.lower())) for sample in data]
            seq_len = int(round(np.percentile(lengths, length_percentile)))
    
        self.max_len = min([seq_len, tokenizer.max_len]) # если max_len токенайзера оказалась меньше заданной, берём её
        self.data = []
        for text in tqdm(data): 
            self.data.append(tokenizer.tokenize(tokenizer.special_tokens_map['bos_token'] + text.lower())[:self.max_len])

        # for i in range(0, len(tokenized_text) - self.max_len + 1, self.max_len):  # Truncate in block of block_size
        #     seq = tok_ids[i: i + self.max_len]
        #     spec_seq = tokenizer.build_inputs_with_special_tokens(seq)
        #     self.examples.append(spec_seq)
        #     print(seq)
        #     print(spec_seq)
        #     print()

        # for i in range(10):
        #     print(self.examples[i])
        # # Note that we are loosing the last truncated example here for the sake of simplicity (no padding)
        # If your dataset is small, first you should loook for a bigger one :-) and second you
        # can change this behavior by adding (model specific) padding.

    def __len__(self):
        return len(self.data)


    def __getitem__(self, item):
        x_toks = self.data[item]
        y_toks = x_toks[1:] + [tokenizer.special_tokens_map['eos_token']]
        assert len(x_toks) == len(y_toks)
        # print(x_toks)
        # print(y_toks)

        x_ids = tokenizer.encode(x_toks, max_length=self.max_len, pad_to_max_length=True)
        y_ids = tokenizer.encode(y_toks, max_length=self.max_len, pad_to_max_length=True)

        # print(x_ids)
        # print(y_ids)

        x = torch.tensor(x_ids).long()
        y = torch.tensor(y_ids).long()

        # Я ни в чём не уверена, но в сберовском pretrain_transformers почему-то нашла такое (метки -- просто копия инпутов), 
        # а в мегатроновском скрипте, на котором он, насколько я поняла, основан, метки были смещённые. Решила делать как в исходнике -- может быть зря, не знаю
        # https://github.com/sberbank-ai/ru-gpts/blob/8234af5cce96cd766f07a461ed5d44957649376e/pretrain_transformers.py#L173 
        # https://github.com/sberbank-ai/ru-gpts/blob/8234af5cce96cd766f07a461ed5d44957649376e/pretrain_megatron.py#L261
        # https://github.com/microsoft/DeepSpeedExamples/blob/78d69cb2f89a27b1e9b072df8c3e47d00c024fdc/Megatron-LM/pretrain_gpt2.py#L265
        # y = x.clone() 
        return x, y

In [17]:
train_dataset = TextDataset(train_corpus, tokenizer)
test_dataset = TextDataset(test_corpus, tokenizer, seq_len=train_dataset.max_len)

In [18]:
# уже на 32 cuda out of memory
train_dataloader = DataLoader(train_dataset, batch_size=16)
test_dataloader = DataLoader(test_dataset, batch_size=16)

In [19]:
for x, y in train_dataloader:
    print(x, x.size())
    # print(y, y.size())
    break

tensor([[    1, 38627,  5352,  ...,     0,     0,     0],
        [    1,  1313, 19147,  ...,     0,     0,     0],
        [    1,   275,  5566,  ...,     0,     0,     0],
        ...,
        [    1,   404,  4491,  ...,   289,   282, 19363],
        [    1, 15169,  2934,  ...,     0,     0,     0],
        [    1, 10148,   349,  ...,     0,     0,     0]]) torch.Size([16, 204])


# Обучение

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [21]:
model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
model.resize_token_embeddings(len(tokenizer))
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [22]:
epochs = 50

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": 0.01},
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(optimizer_grouped_parameters, lr=5e-05, eps=1e-08)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader)//epochs)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=5000, num_training_steps=-1)

In [23]:
metrics = defaultdict(list)

model.zero_grad()
for epoch in range(epochs):

    model.train()
    epoch_train_losses = []
    for x, y in tqdm(train_dataloader, desc='Training epoch {}'.format(epoch+1)):
        x = x.to(device)
        y = y.to(device)

        outputs = model(x, labels=y)
        loss = outputs[0]  # model outputs are always tuple in transformers (see doc)
        loss.backward()

        epoch_train_losses.append(loss.item())

        optimizer.step()
        scheduler.step() 
        optimizer.zero_grad()

    metrics['train_losses'].append(np.mean(epoch_train_losses))

    model.eval()
    epoch_test_losses = []
    for x, y in tqdm(test_dataloader, desc='Testing epoch {}'.format(epoch+1)):
        x = x.to(device)
        y = y.to(device)

        with torch.no_grad():
            outputs = model(x, labels=y)
            loss = outputs[0] 
            epoch_test_losses.append(loss.item())

    metrics['test_losses'].append(np.mean(epoch_test_losses))
    metrics['test_perplexes'].append(torch.exp(torch.tensor(metrics['test_losses'][-1])).item())
    
    print('Epoch {}: Train loss: {} Test loss: {} Test perplex: {}'.format(epoch+1, metrics['train_losses'][-1], metrics['test_losses'][-1], metrics['test_perplexes'][-1]))

    # если тренировочный лосс падает уже две эпохи подряд, а валидационный растёт
    if epoch > 1 and \
    metrics['train_losses'][-1] < metrics['train_losses'][-2] and metrics['train_losses'][-2] < metrics['train_losses'][-3] \
    and \
    metrics['test_losses'][-1] > metrics['test_losses'][-2] and metrics['test_losses'][-2] > metrics['test_losses'][-3]:
        print('Early stopping')
        break


Epoch 1: Train loss: 13.86624978383382 Test loss: 12.92039450009664 Test perplex: 408560.17926599993



Epoch 2: Train loss: 11.934636391533745 Test loss: 11.024150053660074 Test perplex: 61337.706907801716



Epoch 3: Train loss: 10.090535312228733 Test loss: 8.639801263809204 Test perplex: 5652.206414836163



Epoch 4: Train loss: 7.281683275434706 Test loss: 5.795318881670634 Test perplex: 328.7570018507975



Epoch 5: Train loss: 5.266916921403673 Test loss: 4.8079357743263245 Test perplex: 122.47853306299503


...


Epoch 29: Train loss: 3.4814528359307184 Test loss: 3.903671145439148 Test perplex: 49.58414599503491



Epoch 30: Train loss: 3.4533765421973333 Test loss: 3.899291177590688 Test perplex: 49.36744395021419



Epoch 31: Train loss: 3.4284013165367972 Test loss: 3.896335999170939 Test perplex: 49.22176969793305



Epoch 32: Train loss: 3.3991520192888047 Test loss: 3.8987807830174765 Test perplex: 49.34225350380909



Epoch 33: Train loss: 3.369143729739719 Test loss: 3.9033392866452536 Test perplex: 49.56769379020413
Early stopping


In [24]:
torch.save(model.state_dict(), 'model-33')

С грехом пополам, но перплексия чуть ниже 50! Надо было бы, наверное, взять корпус побольше и батчи, тогда получще было бы, но колаб крашился от нехватки памяти.

# Генерация

## Тест

In [25]:
# на батче генерила встроенным методом, и всё было подозрительно хорошо
for x, y in test_dataloader:
    x = x.to(device)
    y = y.to(device)
    output_sequences = model.generate(input_ids=x)
    for one_x, one_y, seq in zip(x, y, output_sequences):
        print("Input:\t{}".format(tokenizer.decode(one_x, skip_special_tokens=False)))
        print("Output:\t{}".format(tokenizer.decode(seq, skip_special_tokens=False)))
        print("Y:\t{}".format(tokenizer.decode(one_y, skip_special_tokens=False)))
        print()
    break

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Input:	<s>уж вечер надвинуться хочет, туман над волнами растет, таинственно море рокочет и что-то, белеясь, встает. из волн поднимается фея и села со мной у зыбей. вздымаются груди, белея под легким покровом у ней. она обняла, охватила, больнее мне всё и больней. «меня ты не в меру сдавила, прекрасная фея морей!» — «тебя я руками сжимаю, насильно в объятиях жму, тобой я согреться желаю в вечернюю хладную мглу». луна всё взирает бледнее с заоблачной выси своей. «твой взор всё мутней и влажнее, прекрасная фея морей!» — «мой взор не влажнее нимало, он мутен, как будто в слезах, — когда я из волн выступала, осталася капля в глазах.» вскричалися чайки нежданно, прибой всколыхался сильней. «стучит твое сердце так странно
Output:	<s>уж вечер надвинуться хочет, туман над волнами растет, таинственно море рокочет и что-то, белеясь, встает. из волн поднимается фея и села со мной у зыбей. вздымаются груди, белея под легким покровом у ней. она обняла, охватила, больнее мне всё и больней. «меня ты н

## Рандомное начало

In [28]:
def encode_srt_input(input):
    toks = tokenizer.tokenize(tokenizer.special_tokens_map['bos_token'] + input.lower().replace('\n', ' '))[:train_dataset.max_len]                        
    ids = tokenizer.encode(toks, max_length=train_dataset.max_len, pad_to_max_length=True)
    return ids

In [29]:
def generate_ids(ids):
    model.eval()
    input = torch.tensor(ids).long()
    input = input.to(device)
    with torch.no_grad():
        pred = model(input)[0]
    # print(pred.size())
    soft = torch.softmax(pred, 1).detach().cpu().numpy()
    # print(soft.shape)
    return soft

In [30]:
def prob2dict(probs):
    return {i:prob for i, prob in enumerate(probs)}

In [31]:
def sort_probs(probs):
    prob_dict = prob2dict(probs)
    return sorted(prob_dict, key=prob_dict.get, reverse=True)
    # return list(prob_dict.items()).sort(key=lambda tup: tup[1], reverse=True) # возвращает None, придётся хранить список в переменной

In [32]:
def softmax_ids(probs, ids):
    # перераспределяем вероятности по топу индексов
    return torch.softmax(torch.tensor([probs[id] for id in ids]), 0).detach().cpu().numpy()

In [33]:
def choose_id_greedy(probs, topn=None):
    # берём индекс самого вероятного
    return np.argmax(probs)

In [34]:
def choose_top_k(probs, topn=5):
    # выбираем индекс согасно вероятности на нём
    # можем выбирать из топа, для этого сортируем
    if topn == -1: # тогда выбираем из всех индексов
        id = np.random.choice([i for i in range(len(probs))], p=probs) 
    else:
        # сортируем и получаем индексы и пересчитываем вероятности для них
        ids = sort_probs(probs)[:topn] 
        probs = softmax_ids(probs, ids)
        # print(probs)
        # print(np.sum(probs))
        id = np.random.choice(ids, p=probs)   
    return id

In [35]:
def choose_equal_k(probs, topn=5):
    # с одинаковой вероятностью выбираем из топа 
    top = sort_probs(probs)[:topn]
    return np.random.choice(top)

In [36]:
def choose_nucleus(probs, p=0.9):
    # суммируем максимальные вероятности, пока не дойдём до порога
    cum_prob = 0
    ids = []
    for id in sort_probs(probs):
        cum_prob+= probs[id]
        ids.append(id)
        if cum_prob >= p:
            break
    # print(len(ids), ids)
    # print(cum_prob)
    probs = softmax_ids(probs, ids)
    return np.random.choice(ids, p=probs)


In [37]:
# pred = generate_ids(encode_srt_input('Осень'))

In [38]:
# choose_id_greedy(pred[0])

In [39]:
# choose_top_k(pred[0], 3)

In [40]:
# choose_equal_k(pred[0], 3)

In [41]:
# choose_nucleus(pred[0], 0.9)

In [42]:
def decode_ids(ids):
    return [tokenizer.decode(id) for id in ids]

In [43]:
def generate(ids, choose_fn, topn=5):
    pred_seq = generate_ids(ids)
    # print(pred_seq.shape)
    print()
    pred_ids = [choose_fn(pred, topn) for pred in pred_seq]
    # print(type(pred_ids[0]))
    print(decode_ids([pred_ids]))
    print(tokenizer.decode(pred_ids, skip_special_tokens=True))
    # return pred_ids

Попробовала генерить разными методами. Ничего хорошего не получилось -- наверное, надо было текстов побольше взять. Наверное, нет смысла пытатья генерить из этого какую-то цепочку до eos

In [44]:
input_str = 'Была пора'
input = encode_srt_input(input_str)
for i in range(1):
    print(input)
    generate(input, choose_id_greedy)
    generate(input, choose_top_k, 5)
    generate(input, choose_equal_k, 5)
    generate(input, choose_nucleus, 0.8)

[1, 38627, 5352, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

[',,,,<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

А если погенерить на целом тексте из трейна, то nucleus samplig сгенерил даже что-то, отдалённо напоминающее текст. А остальные методы ухватили обилие "ой" и "ою", и на том спасибо. В идеале бы добавить какие-то методы доменной адаптации, но я уже не успеваю, к сожалению.

In [45]:
input = encode_srt_input(train_corpus[0])
for i in range(1):
    print(decode_ids(input))
    generate(input, choose_id_greedy)
    generate(input, choose_top_k, 5)
    generate(input, choose_equal_k, 5)
    generate(input, choose_nucleus, 0.7)

['<s>', 'была', ' пора', ' —', ' своей', ' иг', 'рою', ',', ' своею', ' ри', 'з', 'ою', ' ст', 'ально', 'ю', ' морской', ' простор', ' меня', ' плен', 'ял', ';', ' я', ' дорож', 'ил', ' и', ' в', ' ти', 'шь', ' и', ' в', ' бури', ' то', ' нег', 'ой', ' та', 'ющей', ' лаз', 'ури', ',', ' то', ' пен', 'ой', ' у', ' прибреж', 'ных', ' скал', '.', ' но', ' вот', ',', ' о', ' море', ',', ' властью', ' тайной', ' не', ' всё', ' мне', ' мил', ' твой', ' блеск', ' случай', 'ный', ' и', ' в', ' душу', ' прос', 'ится', ' мою', ';', ' див', 'ясь', ' кра', 'се', ' жесток', 'овый', 'ной', ',', ' я', ' перед', ' мощи', 'ю', ' стихий', 'ной', ' в', ' священ', 'ном', ' треп', 'ете', ' стою', '.', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pa

# Cкрипты Сбера (просто чтобы был под рукой образец, как должно было быть)

In [46]:
# !wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py
# !wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

In [47]:
# %%writefile setup.sh

# export CUDA_HOME=/usr/local/cuda-10.1
# git clone https://github.com/NVIDIA/apex
# pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [48]:
# !sh setup.sh

## Prepare data (essays)

In [49]:
# !wget -nc 'https://raw.githubusercontent.com/AnnaSafaryan/DLProject/main/essays.txt'
# text = open('essays.txt', "r").read()
# valid_size = 5
# topics = []
# all_essays = []
# for line in text.split("</s>"):
#     if "Тема:" in line and "Сочинение:" in line:
#         essay_text = line.split("Сочинение:")
#         if len(essay_text) == 2:
#             topic = essay_text[0].replace("<s>", " ").replace("</s>", " ").strip()
#             essay_text = essay_text[1].replace("<s>", " ").replace("</s>", " ").strip()
#             essay_text = f"Сочинение: {essay_text}"
#             essay_res = f"<s>{topic}\n{essay_text}</s>"
#             all_essays.append(essay_res)
#             topics.append(topic)
# random.seed(1234)
# np.random.seed(1234)
# unique_topics = list(set(topics))
# valid_topics = []
# for _ in range(valid_size):
#     # Use randint for more speed (on big lists it is faster)
#     idx = np.random.randint(0, len(unique_topics))
#     valid_topics.append(unique_topics[idx])
# import nltk

# train = []
# valid = []
# for topic, essay in zip(topics, all_essays):
#     is_train = True
#     for valid_topic in valid_topics:
#         if (
#             nltk.edit_distance(valid_topic, topic[:len(valid_topic)]) < 20 or
#             nltk.edit_distance(valid_topic[:len(topic)], topic) < 20 or
#             nltk.edit_distance(valid_topic[len(topic):], topic) < 20 or
#             nltk.edit_distance(valid_topic, topic[len(valid_topic):]) < 20
#             ):
#             is_train = False
#     if is_train:
#         train.append(essay.replace('\n', '\t'))
#     else:
#         valid.append(essay.replace('\n', '\t'))
# len(train), len(valid)
# train[:10]
# open("train.txt", "w", encoding='utf-8').write("\n".join(train))
# open("valid.txt", "w", encoding='utf-8').write("\n".join(valid))

## Run finetuning
The following code download our model and tokenizer from transformers and finetune model essays.

This took aroung ten minutes and obtain perplexity = 13-16

In [50]:
# !python pretrain_transformers.py \
#     --output_dir=essays_model \
#     --model_type=gpt2 \
#     --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
#     --do_train \
#     --train_data_file=train.txt \
#     --do_eval \
#     --fp16 \
#     --eval_data_file=valid.txt \
#     --per_gpu_train_batch_size 1 \
#     --gradient_accumulation_steps 1 \
#     --num_train_epochs 1 \
#     --block_size 2048 \
#     --overwrite_output_dir

## Check our model

In [51]:
# ## Select topic
# for idx in range(len(valid)):
#   if "образует его общество" in valid[idx]:
#     break

In [52]:
# valid[idx]

In [53]:
# !python generate_transformers.py \
#     --model_type=gpt2 \
#     --model_name_or_path=essays_model \
#     --k=5 \
#     --p=0.95 \
#     --length=500